In [1]:
import firebase_admin 
from firebase_admin import credentials, storage
from firebase_admin import db
import os
import pandas as pd
import dotenv


dotenv.load_dotenv()

True

In [2]:
service_account_info = {
    "type": os.getenv("FIREBASE_TYPE"),
    "project_id": os.getenv("FIREBASE_PROJECT_ID"),
    "private_key_id": os.getenv("FIREBASE_PRIVATE_KEY_ID"),
    "private_key": os.getenv("FIREBASE_PRIVATE_KEY").replace('\\n', '\n'),
    "client_email": os.getenv("FIREBASE_CLIENT_EMAIL"),
    "client_id": os.getenv("FIREBASE_CLIENT_ID"),
    "auth_uri": os.getenv("FIREBASE_AUTH_URI"),
    "token_uri": os.getenv("FIREBASE_TOKEN_URI"),
    "auth_provider_x509_cert_url": os.getenv("FIREBASE_AUTH_PROVIDER_X509_CERT_URL"),
    "client_x509_cert_url": os.getenv("FIREBASE_CLIENT_X509_CERT_URL"),
    "universe_domain": os.getenv("FIREBASE_UNIVERSE_DOMAIN")
}


In [3]:
import firebase_admin
from firebase_admin import credentials, db, storage

if not firebase_admin._apps:
    cred = credentials.Certificate(service_account_info)
    firebase_admin.initialize_app(cred, {
        'databaseURL': "https://fullstack-ai-coffee-default-rtdb.firebaseio.com",
        'storageBucket': os.getenv("FIREBASE_STORAGE_BUCKET")
    })


In [4]:
bucket = storage.bucket(os.getenv("FIREBASE_STORAGE_BUCKET"))

In [ ]:
image_folder_path = './Products/Images'

In [34]:
print("firebase_admin._apps =", firebase_admin._apps)

firebase_admin._apps = {'[DEFAULT]': <firebase_admin.App object at 0x122e19210>}


In [ ]:
products_collection = db.reference('products')

In [7]:
print("Products collection reference:", products_collection)

Products collection reference: <firebase_admin.db.Reference object at 0x10c241090>


In [8]:
df = pd.read_json('products/products.jsonl',lines=True)
df.head(2)

,name,category,description,ingredients,price,rating,image_path
0,Cappuccino,Coffee,A rich and creamy cappuccino made with freshly...,"[Espresso, Steamed Milk, Milk Foam]",4.50,4.7,cappuccino.jpg
1,Jumbo Savory Scone,Bakery,"Deliciously flaky and buttery, this jumbo savo...","[Flour, Butter, Cheese, Herbs, Baking Powder, ...",3.25,4.3,SavoryScone.webp


In [11]:
def upload_to_firebase(bucket, image_path):
    image_name = image_path.split('/')[-1]
    blob = bucket.blob(f'products/images/{image_name}')
    blob.upload_from_filename(image_path)
    blob.make_public()
    return blob.public_url

In [ ]:
df.head(2)

,name,category,description,ingredients,price,rating,image_path
0,Cappuccino,Coffee,A rich and creamy cappuccino made with freshly...,"[Espresso, Steamed Milk, Milk Foam]",4.50,4.7,cappuccino.jpg
1,Jumbo Savory Scone,Bakery,"Deliciously flaky and buttery, this jumbo savo...","[Flour, Butter, Cheese, Herbs, Baking Powder, ...",3.25,4.3,SavoryScone.webp


#### We want to loop through each product in the dataset, uploaded its image to Firebase Storage, and got a public image URL. Then we prepared the product data by removing the local image path and adding the new image URL. Finally, we pushed this cleaned product data into the Firebase Realtime Database under the products node.

In [14]:
for index, row in df.iterrows():
    print(index, row['name'])

    image_path = os.path.join(image_folder_path, row['image_path'])
    
    image_url = upload_to_firebase(bucket, image_path)
    product_data = row.to_dict()
    product_data.pop('image_path')
    product_data['image_url'] = image_url

    products_collection.push().set(product_data)


0 Cappuccino
1 Jumbo Savory Scone
2 Latte
3 Chocolate Chip Biscotti
4 Espresso shot
5 Hazelnut Biscotti
6 Chocolate Croissant
7 Dark chocolate
8 Cranberry Scone
9 Croissant
10 Almond Croissant
11 Ginger Biscotti
12 Oatmeal Scone
13 Ginger Scone
14 Chocolate syrup
15 Hazelnut syrup
16 Carmel syrup
17 Sugar Free Vanilla syrup
